In [4]:
import google.generativeai as genai
import textwrap
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def to_markdown(text):
    text = text.replace('•', '  *')
    return textwrap.indent(text, '> ', predicate=lambda _: True)

def get_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    text = ' '.join([para.get_text() for para in paragraphs])
    return text

def split_text(text, chunk_size=1000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks


/home/gcasado0/proyectos/chatbot/chatbot/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Used to securely store your API key
import os

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
print(GOOGLE_API_KEY)

/home/gcasado0


In [ ]:
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')

In [20]:
# Lista de URLs
urls = [
    "https://www.rosario.gob.ar/inicio/licencia-de-conducir",
    "https://www.rosario.gob.ar/inicio/duplicado-de-licencia-de-conducir",
    "https://www.rosario.gob.ar/inicio/nueva-licencia-de-conducir",
    "https://www.rosario.gob.ar/inicio/renovacion-de-licencia-de-conducir",
    "https://www.rosario.gob.ar/inicio/renovacion-de-licencia-de-conducir-con-ampliacion"
]

# Obtener y dividir el contenido de las páginas web
documents = []
for url in urls:
    text = get_text_from_url(url)
    chunks = split_text(text)
    documents.extend(chunks)

In [8]:
# Crear un índice de búsqueda usando TF-IDF
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(documents).toarray()

In [9]:
def retrieve_relevant_docs(query, k=5):
    #query_vec = vectorizer.transform([query]).toarray()
    query_vec = vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, vectors).flatten()
    indices = np.argsort(similarities)[-k:]
    return [documents[i] for i in indices]


In [22]:
# Consultar el índice y generar una respuesta
query = "extravie mi licencia que tengo que llevar para hacer el duplicado? responder en español"
relevant_docs = retrieve_relevant_docs(query)
context = " ".join(relevant_docs)

response = model.generate_content(f"Contexto: {context}\n\nPregunta: {query}")
markdown_response = to_markdown(response.text)
print(markdown_response)


> Para el duplicado de tu licencia de conducir por extravío, necesitarás llevar la siguiente documentación al momento del turno (15 minutos antes):
> 
> * **Declaración jurada por extravío:** (Debes descargarla y completarla previamente).
> * **DNI.**
> * **Certificado de Legalidad en original:** De la autoridad que emitió la licencia, con sellos oficiales, membrete, firma y sello de un personal jerárquico.  (Esto evita la leyenda "Principiante" y sirve como antecedente si tienes categoría profesional).
> * **Boletas abonadas:**  CeNAT ($6.800) y sellado municipal (monto variable según el tiempo transcurrido desde el último psicofísico).  Recuerda que el pago se realiza por separado a pesar de que la boleta sea una sola.
> 
> 
> Además, según tu situación y si corresponde a lo establecido en la normativa vigente, deberás llevar otras documentaciones o realizar otros trámites (psicofísico, curso teórico, exámenes teórico y práctico) que se detallan en la información proporcionada.  Recu

In [13]:
# Consultar el índice y generar una respuesta
query = "Cual es el costo de renovar una licencia? responder en español"
relevant_docs = retrieve_relevant_docs(query)
context = " ".join(relevant_docs)

response = model.generate_content(f"Contexto: {context}\n\nPregunta: {query}")
markdown_response = to_markdown(response.text)
print(markdown_response)

> El costo de renovar una licencia es de $6.800.  Sin embargo, el texto indica que este precio puede variar según la duración de la licencia.



In [21]:
# Consultar el índice y generar una respuesta
query = "Como hago para pagar el cenat? responder en español"
relevant_docs = retrieve_relevant_docs(query)
context = " ".join(relevant_docs)
print(context)

response = model.generate_content(f"Contexto: {context}\n\nPregunta: {query}")
markdown_response = to_markdown(response.text)
display(markdown_response)

Este trámite se realiza si querés agregar o quitar clases a una licencia vigente emitida por la Provincia de Santa Fe o si debés renovarla, modificando las clases en ese mismo momento. Para realizar trámites online en la Muni es necesario que tengas Perfil Digital. Si todavía no tenes una cuenta, registrate. Si cumplis con los requisitos: Agendá un turno Una vez agendado, te va a llegar un mail como constancia de turno en el que vas a tener a mano lo que necesitás saber para asistir el día del examen. El próximo paso es abonar los sellados. Tiene un costo de $6.800 El costo varía según la duración de la licencia. El día del turno presentate 15 minutos antes con la siguiente documentación: Solo si corresponde: De manera virtual De manera presencial Tené en cuenta que la boleta es una sola, pero contiene dos códigos de barra que se abonan por separado: sellado municipal y colegio de médicos. De manera virtual De manera presencial Una vez que apruebes el psicofísico y antes de rendir el e

'> El texto indica que el costo del Certificado Nacional de Antecedentes de Tránsito (CeNAT) es de $6.800  para duplicados de licencia si pasó menos de un año desde el último psicofísico.  Sin embargo, **el texto no especifica cómo pagar el CeNAT**.  Para saber cómo abonarlo, deberías consultar la página web de la municipalidad de Rosario o comunicarse con ellos directamente.\n'